Reference: https://geodacenter.github.io/workbook/6a_local_auto/lab6a.html#local-geary

# Univariate local geary

Interestingly, following [this equation](https://www.biomedware.com/files/documentation/spacestat/Statistics/Gearys_C/Geary_s_C_statistic.htm) which explicitly calls for standardization of input data. We also do NOT divide by 2.

$$ c_i = \sum_j w_{ij} (z_i - z_j)^2 $$ 

where: 

$z_i = x_i - \bar{x}$ and $z_j = x_j - \bar{x}$, and $w_{ij}$ are the elements of the row-standardized binary symmetric spatial weight matrix W. 

or, $$ c_i = (1/m^2) * \sum_j w_{ij} (x_i - x_j)^2 $$

where,

$$ m^2 = \sum_i (x_i−\bar{x})^2/n $$

## Load in example data

In [1]:
import libpysal as lp
import geopandas as gpd
from scipy import stats
import numpy as np
guerry = lp.examples.load_example('Guerry')
guerry_ds = gpd.read_file(guerry.get_path('Guerry.shp'))

In [2]:
wq = lp.weights.Queen.from_dataframe(guerry_ds)

In [3]:
wq[0]

{66: 1.0, 35: 1.0, 68: 1.0, 36: 1.0}

In [4]:
wq.transform = 'r'
wq[0]

{66: 0.25, 35: 0.25, 68: 0.25, 36: 0.25}

In [5]:
x = guerry_ds['Donatns']

print("x_i is", x[0])
print("x_j are", x[66], x[35], x[68], x[36])

x_i is 5098
x_j are 1983 4077 3710 3012


In [6]:
# Calculate score
zscore_x = (x - np.mean(x))/np.std(x)
zscore_x

0    -0.336188
1     0.450441
2     0.879023
3    -0.825375
4     0.049370
        ...   
80    1.512380
81    0.454785
82    1.467288
83   -0.555029
84   -0.506214
Name: Donatns, Length: 85, dtype: float64

# Build observed local geary values

In [7]:
adj_list = wq.to_adjlist(remove_symmetric=False)
adj_list.head()

,focal,neighbor,weight
0,0,66,0.250000
1,0,35,0.250000
2,0,68,0.250000
3,0,36,0.250000
4,1,48,0.166667


In [8]:
import pandas as pd
zseries = pd.Series(zscore_x, index=wq.id_order)
zseries[0:5]

0   -0.336188
1    0.450441
2    0.879023
3   -0.825375
4    0.049370
Name: Donatns, dtype: float64

In [9]:
# Define z_i
zi = zseries.loc[adj_list.focal].values
zi[0:5]

array([-0.33618783, -0.33618783, -0.33618783, -0.33618783,  0.45044136])

In [10]:
# Define zj
zj = zseries.loc[adj_list.neighbor].values
zj[0:5]

array([-0.98050808, -0.54737594, -0.62328783, -0.76766521, -0.5709562 ])

In [11]:
(zi-zj)[0:5]

array([0.64432025, 0.21118812, 0.2871    , 0.43147738, 1.02139756])

In [12]:
(zi-zj)**2

array([4.15148590e-01, 4.46004201e-02, 8.24264123e-02, 1.86172733e-01,
       1.04325299e+00, 2.67617709e-01, 5.53872475e-01, 4.94589963e-01,
       3.37591065e-01, 6.63159957e-01, 8.53015374e-02, 2.25693761e+00,
       2.46439290e-05, 1.16134943e-02, 2.42399264e+00, 1.07389771e+00,
       7.65178635e-01, 9.46039788e-02, 3.45083461e-03, 3.94302864e-04,
       3.56105843e-01, 7.65178635e-01, 7.30833213e-01, 3.38135670e-02,
       5.51412189e-03, 2.84791404e-03, 8.35857038e-03, 8.38578139e-04,
       5.63859940e-02, 1.61356296e-01, 2.54096407e-01, 2.67617709e-01,
       2.07831152e-01, 3.94302864e-02, 6.74942450e-02, 2.80848393e+00,
       5.39192908e-03, 7.01774384e-03, 4.88011403e-02, 1.90915149e-02,
       1.22921222e-01, 3.48097208e-02, 3.94302864e-02, 3.21607124e-02,
       3.74861542e-03, 3.51346459e+00, 3.32831663e-02, 7.76287613e-02,
       1.13674401e-03, 1.68241408e-01, 2.42349082e-03, 5.86679962e-02,
       1.00285431e-01, 2.30504594e-02, 7.79749315e-04, 4.88011403e-02,
      

Multiply by spatial weights

In [13]:
sum(list(wq.weights.values()), []) * (zi-zj)**2

array([1.03787147e-01, 1.11501050e-02, 2.06066031e-02, 4.65431831e-02,
       1.73875498e-01, 4.46029515e-02, 9.23120791e-02, 8.24316606e-02,
       5.62651775e-02, 1.10526659e-01, 1.42169229e-02, 3.76156268e-01,
       4.10732150e-06, 1.93558239e-03, 4.03998774e-01, 1.78982952e-01,
       1.91294659e-01, 2.36509947e-02, 8.62708652e-04, 9.85757159e-05,
       1.18701948e-01, 2.55059545e-01, 2.43611071e-01, 4.83050957e-03,
       7.87731699e-04, 4.06844863e-04, 1.19408148e-03, 1.19796877e-04,
       8.05514201e-03, 2.30508994e-02, 8.46988022e-02, 8.92059030e-02,
       6.92770507e-02, 1.31434288e-02, 2.24980817e-02, 9.36161309e-01,
       1.07838582e-03, 1.40354877e-03, 9.76022807e-03, 3.81830297e-03,
       2.45842445e-02, 6.96194416e-03, 7.88605727e-03, 6.43214247e-03,
       7.49723084e-04, 7.02692918e-01, 4.75473805e-03, 1.10898230e-02,
       1.62392001e-04, 2.40344868e-02, 3.46212974e-04, 8.38114231e-03,
       1.43264902e-02, 7.68348646e-03, 2.59916438e-04, 1.62670468e-02,
      

In [14]:
test = sum(list(wq.weights.values()), []) * (zi-zj)**2

In [15]:
test 

array([1.03787147e-01, 1.11501050e-02, 2.06066031e-02, 4.65431831e-02,
       1.73875498e-01, 4.46029515e-02, 9.23120791e-02, 8.24316606e-02,
       5.62651775e-02, 1.10526659e-01, 1.42169229e-02, 3.76156268e-01,
       4.10732150e-06, 1.93558239e-03, 4.03998774e-01, 1.78982952e-01,
       1.91294659e-01, 2.36509947e-02, 8.62708652e-04, 9.85757159e-05,
       1.18701948e-01, 2.55059545e-01, 2.43611071e-01, 4.83050957e-03,
       7.87731699e-04, 4.06844863e-04, 1.19408148e-03, 1.19796877e-04,
       8.05514201e-03, 2.30508994e-02, 8.46988022e-02, 8.92059030e-02,
       6.92770507e-02, 1.31434288e-02, 2.24980817e-02, 9.36161309e-01,
       1.07838582e-03, 1.40354877e-03, 9.76022807e-03, 3.81830297e-03,
       2.45842445e-02, 6.96194416e-03, 7.88605727e-03, 6.43214247e-03,
       7.49723084e-04, 7.02692918e-01, 4.75473805e-03, 1.10898230e-02,
       1.62392001e-04, 2.40344868e-02, 3.46212974e-04, 8.38114231e-03,
       1.43264902e-02, 7.68348646e-03, 2.59916438e-04, 1.62670468e-02,
      

In [16]:
# Create a df that uses the adjacency list focal values and the BBs counts
temp = pd.DataFrame(adj_list.focal.values, test).reset_index()
temp

,index,0
0,0.103787,0
1,0.011150,0
2,0.020607,0
3,0.046543,0
4,0.173875,1
...,...,...
415,0.025788,84
416,0.326386,84
417,0.003818,84
418,0.001947,84


In [17]:
# Temporarily rename the columns
temp.columns = ['Eij', 'ID']
temp = temp.groupby(by='ID').sum()

In [18]:
temp.Eij.values[0:5]

array([0.18208704, 0.56001403, 0.97529461, 0.21590694, 0.61737256])

# Start building function

In [19]:
import numpy as np
import pandas as pd
import warnings
from scipy import sparse
from scipy import stats
from sklearn.base import BaseEstimator
import libpysal as lp
from esda.crand import (
    crand as _crand_plus,
    njit as _njit,
    _prepare_univariate
)



class Local_Geary(BaseEstimator):
    """Local Geary - Univariate"""

    def __init__(self, connectivity=None, inference=None):
        """
        Initialize a Local_Geary estimator

        Arguments
        ---------
        connectivity     : scipy.sparse matrix object
                           the connectivity structure describing the
                           relationships between observed units.
        inference        : str
                           describes type of inference to be used. options are
                           "chi-square" or "permutation" methods.

        Attributes
        ----------
        localG           : numpy array
                           Array of Local Geary values for each spatial unit.
        pval             : numpy array
                           P-values for inference based on either
                           "chi-square" or "permutation" methods.
        """

        self.connectivity = connectivity
        self.inference = inference

    def fit(self, x):
        """
        Arguments
        ---------
        x                : numpy.ndarray
                           array containing continuous data

        Returns
        -------
        the fitted estimator.

        Notes
        -----
        Technical details and derivations can be found in :cite:`Anselin1995`.

        Examples
        --------
        Guerry data replication GeoDa tutorial
        >>> import libpysal
        >>> import geopandas as gpd
        >>> guerry = lp.examples.load_example('Guerry')
        >>> guerry_ds = gpd.read_file(guerry.get_path('Guerry.shp'))
        >>> w = libpysal.weights.Queen.from_dataframe(guerry_ds)
        """
        x = np.asarray(x).flatten()

        w = self.connectivity
        w.transform = 'r'

        self.localG = self._statistic(x, w)

        if self.inference is None:
        #   self.p_sim, self.rjoins = _crand_plus(
        #       z=self.x, 
        #       w=self.w, 
        #       observed=self.localG,
        #       permutations=permutations, 
        #       keep=True, 
        #       n_jobs=n_jobs,
        #       stat_func=_local_geary
        #   )
        #   
            pass
        else:
            raise NotImplementedError(f'The requested inference method \
            ({self.inference}) is not currently supported!')

        return self

    @staticmethod
    def _statistic(x, w):
        # Caclulate z-scores for x
        zscore_x = (x - np.mean(x))/np.std(x)
        # Create focal (xi) and neighbor (zi) values
        adj_list = w.to_adjlist(remove_symmetric=False)
        zseries = pd.Series(zscore_x, index=wq.id_order)
        zi = zseries.loc[adj_list.focal].values
        zj = zseries.loc[adj_list.neighbor].values
        # Carry out local Geary calculation
        gs = sum(list(wq.weights.values()), []) * (zi-zj)**2
        # Reorganize data
        adj_list_gs = pd.DataFrame(adj_list.focal.values, gs).reset_index()
        adj_list_gs.columns = ['gs', 'ID']
        adj_list_gs = adj_list_gs.groupby(by='ID').sum()
        
        localG = adj_list_gs.gs.values
        
        return (localG)

# --------------------------------------------------------------
# Conditional Randomization Function Implementations
# --------------------------------------------------------------

@_njit(fastmath=True)
def _local_geary(i, z, permuted_ids, weights_i, scaling):
    zi, zrand = _prepare_univariate(i, z, permuted_ids, weights_i)
    return zi * (zrand @ weights_i)

In [20]:
functest = Local_Geary(connectivity=wq).fit(x)
functest.localG

array([1.82087039e-01, 5.60014026e-01, 9.75294606e-01, 2.15906938e-01,
       6.17372564e-01, 3.84450059e-02, 2.43181756e-01, 9.71802819e-01,
       4.06447101e-02, 7.24722785e-01, 6.30952854e-02, 2.42104497e-02,
       1.59496916e+01, 9.29326006e-01, 9.65188634e-01, 1.32383286e+00,
       3.31775497e-01, 2.99446505e+00, 9.43946814e-01, 2.99570159e+00,
       3.66702291e-01, 2.09592365e+00, 1.46515861e+00, 1.82118455e-01,
       3.10216680e+00, 5.43063937e-01, 5.74532559e+00, 4.79160197e-02,
       1.58993089e-01, 7.18327253e-01, 1.24297849e+00, 8.72629331e-02,
       7.52809650e-01, 4.56515485e-01, 3.86766562e-01, 1.17632604e-01,
       6.90884685e-01, 2.87206102e+00, 4.10455112e-01, 4.04349959e-01,
       1.14211758e-01, 9.59519953e-01, 3.51347976e-01, 7.30240974e-01,
       4.40370938e-01, 7.20360356e-02, 1.66241706e+00, 5.83258909e+00,
       2.30332507e-01, 4.38369688e-01, 8.41461470e-01, 1.52959486e+00,
       4.32157479e-02, 2.08325903e+00, 1.19722984e+00, 1.28169257e+00,
      

## Start working on inference

In [21]:
rowsum = np.array(wq.sparse.sum(axis=1)).flatten()
# non row standardized
Eci = (2 * len(x) * rowsum) / (len(x) - 1)
Eci

# row standardized
Eci = (2*len(x)) * (len(x)-1) / ((len(x)-1)*len(x))
Eci

2.0

### 'Old' method

In [22]:
permutations = 99
xs = [(np.random.permutation(x)) for i in range(permutations)]
results = [Local_Geary(connectivity=wq).fit(xs[i]) for i in range(permutations)]
sim = [results[i].localG for i in range(permutations)]
above = sim >= functest.localG
larger = above.sum(0)
low_extreme = (permutations - larger) < larger
larger[low_extreme] = permutations - larger[low_extreme]
p_sim = (larger + 1.0) / (permutations + 1.0)
p_sim

array([0.09, 0.2 , 0.35, 0.14, 0.37, 0.03, 0.19, 0.46, 0.01, 0.3 , 0.01,
       0.01, 0.01, 0.37, 0.42, 0.4 , 0.1 , 0.22, 0.29, 0.14, 0.11, 0.28,
       0.44, 0.09, 0.15, 0.23, 0.07, 0.01, 0.06, 0.28, 0.46, 0.07, 0.32,
       0.17, 0.15, 0.03, 0.26, 0.2 , 0.1 , 0.17, 0.07, 0.46, 0.08, 0.3 ,
       0.15, 0.06, 0.37, 0.12, 0.05, 0.1 , 0.33, 0.43, 0.01, 0.28, 0.47,
       0.49, 0.16, 0.15, 0.2 , 0.06, 0.49, 0.47, 0.31, 0.18, 0.23, 0.45,
       0.11, 0.25, 0.22, 0.11, 0.07, 0.32, 0.01, 0.18, 0.32, 0.16, 0.01,
       0.01, 0.02, 0.03, 0.44, 0.44, 0.33, 0.42, 0.16])

These p-values aren't lining up with GeoDa...

### 'New' `_crand()` engine

In [23]:
from esda.crand import (
    crand as _crand_plus,
    njit as _njit,
    _prepare_univariate
)

In [24]:
@_njit(fastmath=True)
def _local_geary(i, z, permuted_ids, weights_i, scaling):
    zi, zrand = _prepare_univariate(i, z, permuted_ids, weights_i)
    print(zi)
    print(zrand @ weights_i)
    return zi * (zrand @ weights_i)

In [25]:
p_sim, rG = _crand_plus(z=np.array(x, dtype='float'), w=wq, observed=np.array(functest.localG), 
            permutations=999, keep=True, n_jobs=1, 
            stat_func=_local_geary)

print(p_sim)
print(rG)

5098.0
[12785.75  9300.   12158.25  8017.    5870.5   4204.75  6320.25  9598.5
  5210.75 15421.5   6553.    4992.5   4019.75  9267.25  7700.75  7502.
  3743.25  9848.    6071.5   4409.75  3286.    4586.25  4651.5   8216.25
  5921.5   3888.    8794.25  5968.    4252.    5316.25  3871.75  4491.75
  8190.75  4644.25  4827.5   5185.25  6209.5   8943.75  5858.    3269.
  7866.    6843.    7388.    4506.75  9311.5   3259.5   6795.25  4642.25
  4955.25  6985.75  5533.    6567.25 10883.5   6170.75  9258.    8102.5
  4672.25  4160.   10522.25  5199.75 11644.75  4273.75  6482.25  7431.25
  8945.25  4154.5   9529.25 10471.   10591.    6116.   12123.25  7485.5
 11410.    3861.    6317.25  3312.    7291.5   4045.5   6403.    2829.
  4662.75  6169.5   5561.25  7202.75  6693.75 13932.    9439.    6532.5
  5869.25  6667.25  4169.5   5471.25  3778.    8230.    5389.75  5490.25
  5909.75 10868.5   4288.    9588.75  4933.    7782.5   4377.75  7783.5
  6334.25  7464.    3617.75  7221.75  5179.25  2644.   

[ 6372.14285714  6025.14285714  6000.28571429  9175.42857143
  6409.71428571  3633.85714286  6453.          8884.71428571
  4501.14285714  7467.          6928.42857143  5758.85714286
  4076.85714286  8159.85714286  7024.71428571  7739.
  5440.71428571  8123.14285714  5298.          6162.71428571
  4396.85714286  4547.28571429  6339.42857143  6169.28571429
  5680.28571429  8479.85714286  8030.42857143  5299.85714286
  4185.85714286 11467.          4059.57142857  6820.
  7861.85714286  4150.          4236.57142857  6325.
  6446.          8912.71428571  4824.28571429  7065.71428571
  8202.          7569.28571429  6776.28571429  4176.85714286
 10025.42857143  3719.          7026.85714286  5376.85714286
  4699.          7696.          7755.14285714  8002.71428571
  4287.57142857  6771.28571429  6955.42857143  7039.57142857
  5571.42857143  4521.57142857  7612.57142857  6947.85714286
  9676.57142857  4232.          9872.          8436.
 10343.14285714  9440.71428571  7246.57142857  9311.1428

 11316.    7355.    4495.25 10278.75  5054.75  7560.75  3763.75]
5626.0
[ 6477.5         6352.          6211.         10454.83333333
  7399.5         8136.16666667  4702.5         8026.33333333
  4057.5         6825.66666667  5563.          5878.83333333
  4399.83333333  7979.5         4735.66666667  8028.33333333
  5280.83333333  7807.16666667  5679.          4811.5
  3663.33333333  4348.5         3654.5         7482.16666667
  6420.         10798.16666667  6916.16666667  7046.33333333
  5525.33333333  8094.33333333  4262.5         9035.
  7120.33333333  5633.83333333  6432.33333333  5613.66666667
  7331.5         8446.16666667  6386.5         6619.
  9673.16666667  6673.83333333  6365.33333333  4720.
 10415.33333333  3466.83333333  8917.5         5427.
  5513.          5667.          5146.83333333  7648.33333333
  6249.16666667  6590.66666667  3923.33333333  7874.66666667
  6819.16666667  5724.          7052.5         5211.
  7119.66666667  4353.66666667  9583.83333333  5825.83333333

[ 3436.  11689.   3619.5  8024.5  9949.   7380.5  2073.5 11581.5  3840.5
 12651.5  4073.   2544.   3530.5  5298.   7444.5  9659.   7092.  15714.
  7229.5  3026.5  3058.5  2364.5  2346.   9483.   9022.   9653.5  5128.5
  2867.5  3825.   2444.   3009.   5764.   7213.5  9030.   8047.5  2561.
 10477.  14144.5  8194.   5824.5  4320.   6161.5  5008.   5138.5  7324.
  3653.   9947.5  6212.   4558.  10419.5  3122.   7103.5  3177.   2972.
  3281.   4415.   4020.   5765.   3652.   8033.   4873.5  4873.5  4892.5
 10470.  10108.   9146.   2261.5  4555.5  3771.   5426.5  4296.5  9999.
 11524.5  3688.   8692.   3247.   3770.5  3272.5  4219.5  3609.   4071.
  8207.5  9280.   9814.5  6910.5  8261.5 12027.   7667.   7102.   4060.
  8308.5  5260.5  4571.  19108.5  3626.   3184.5  4930.5 14755.   3270.5
 11857.   7382.  14170.5  8470.5  9516.5  7962.   3571.5  4720.  15974.5
  2705.5  3867.5  7072.   6498.5  6932.5  3491.5  4628.   6224.   2903.
  4794.5  2867.5 14562.   6425.   4143.5  9801.5  3058.5  7

  3584.4  6660.2 11773.2  7924.2  5534.   9503.   5148.2  7490.2  4822.8]
3710.0
[ 6150.71428571  6521.85714286  5391.42857143 10613.14285714
  7857.71428571  7564.71428571  5856.57142857  7120.71428571
  6326.85714286  5734.          6767.14285714  4941.85714286
  5066.71428571  6711.85714286  3917.          7529.42857143
  5963.14285714  7362.14285714  5316.28571429  4657.28571429
  3392.28571429  5715.42857143  4842.          6793.14285714
  5454.28571429  9095.14285714  6451.42857143  7226.85714286
  6069.28571429  7635.71428571  6809.42857143  9373.57142857
  6276.57142857  5776.28571429  6317.14285714  4965.42857143
  6358.85714286  8761.28571429  5048.85714286  6858.85714286
  5260.28571429  7606.          6826.57142857  7122.57142857
  7312.28571429  4698.57142857  8136.28571429  5083.28571429
  5074.28571429  6532.85714286  4064.14285714  5927.57142857
  5600.28571429  5811.14285714  4002.85714286  7411.85714286
  6230.28571429  4698.28571429  6543.14285714  5899.57142857
  79

 11316.    9043.    3142.5  10001.5   4944.5   5737.5   5393.5 ]
8922.0
[ 5651.16666667  6223.83333333  5782.         11315.33333333
  7215.33333333  8252.83333333  4065.5         6168.83333333
  5429.33333333  5408.66666667  7147.33333333  6555.16666667
  4844.5         5133.66666667  4098.33333333  7503.33333333
  5890.33333333  5770.5         4906.          3177.5
  3385.          4986.5         4020.66666667  6858.66666667
  5554.          9898.33333333  6096.33333333  7298.83333333
  6509.33333333  7192.5         7369.5         8995.16666667
  6992.16666667  5751.83333333  6684.83333333  5218.66666667
  4838.          9750.          5210.83333333  8269.5
  5355.83333333  8298.83333333  4916.          6956.83333333
  8251.          4490.16666667  9769.66666667  5201.5
  6375.83333333  5412.66666667  3939.83333333  6201.
  5913.          5572.16666667  4229.33333333  7801.16666667
  6733.5         5132.5         5804.83333333  6617.66666667
  5104.16666667  4779.          8935.83333

# Multivariate Local Geary

$$ c_i = \sum_{h=1}^m \sum_j w_{ij} (x_{hi} - x_{hj})^2 $$

Load in the sample data

In [26]:
import libpysal as lp
import geopandas as gpd
from scipy import stats
import pandas as pd
guerry = lp.examples.load_example('Guerry')
guerry_ds = gpd.read_file(guerry.get_path('Guerry.shp'))

In [27]:
wq = lp.weights.Queen.from_dataframe(guerry_ds)

In [28]:
x = guerry_ds['Donatns']
y = guerry_ds['Suicids']

In [29]:
variables = [x,y]
variables

[0      5098
 1      8901
 2     10973
 3      2733
 4      6962
       ...  
 80    14035
 81     8922
 82    13817
 83     4040
 84     4276
 Name: Donatns, Length: 85, dtype: int64,
 0      35039
 1      12831
 2     114121
 3      14238
 4      16171
        ...  
 80     67963
 81     21851
 82     33497
 83     33029
 84     12789
 Name: Suicids, Length: 85, dtype: int64]

Standardize each variable

In [30]:
from scipy import stats
zseries = [stats.zscore(i) for i in variables]

Build the adj lists

In [31]:
adj_list = wq.to_adjlist(remove_symmetric=False)

In [32]:
# The zseries
zseries = [pd.Series(i, index=wq.id_order) for i in zseries]

In [33]:
low_extreme = (permutations - larger) < larger
larger[low_extreme] = permutations - larger[low_extreme]
p_sim = (larger + 1.0) / (permutations + 1.0)
p_sim

array([0.09, 0.2 , 0.35, 0.14, 0.37, 0.03, 0.19, 0.46, 0.01, 0.3 , 0.01,
       0.01, 0.01, 0.37, 0.42, 0.4 , 0.1 , 0.22, 0.29, 0.14, 0.11, 0.28,
       0.44, 0.09, 0.15, 0.23, 0.07, 0.01, 0.06, 0.28, 0.46, 0.07, 0.32,
       0.17, 0.15, 0.03, 0.26, 0.2 , 0.1 , 0.17, 0.07, 0.46, 0.08, 0.3 ,
       0.15, 0.06, 0.37, 0.12, 0.05, 0.1 , 0.33, 0.43, 0.01, 0.28, 0.47,
       0.49, 0.16, 0.15, 0.2 , 0.06, 0.49, 0.47, 0.31, 0.18, 0.23, 0.45,
       0.11, 0.25, 0.22, 0.11, 0.07, 0.32, 0.01, 0.18, 0.32, 0.16, 0.01,
       0.01, 0.02, 0.03, 0.44, 0.44, 0.33, 0.42, 0.16])

In [34]:
zseries

[0    -0.336188
 1     0.450441
 2     0.879023
 3    -0.825375
 4     0.049370
         ...   
 80    1.512380
 81    0.454785
 82    1.467288
 83   -0.555029
 84   -0.506214
 Length: 85, dtype: float64,
 0    -0.047195
 1    -0.756433
 2     2.478379
 3    -0.711499
 4    -0.649766
         ...   
 80    1.004270
 81   -0.468369
 82   -0.096441
 83   -0.111387
 84   -0.757774
 Length: 85, dtype: float64]

In [35]:
# The focal values
focal = [zseries[i].loc[adj_list.focal].values for
         i in range(len(variables))]
# The neighbor values
neighbor = [zseries[i].loc[adj_list.neighbor].values for
            i in range(len(variables))]

In [37]:
gs = sum(list(wq.weights.values()), []) * (np.array(focal) - np.array(neighbor))**2

In [38]:
temp = pd.DataFrame(gs).T

In [39]:
temp['ID'] = adj_list.focal.values

In [40]:
adj_list_gs = temp.groupby(by='ID').sum()
adj_list_gs.head()

,0,1
ID,,
0,0.728348,0.502200
1,3.360084,0.282630
2,5.851768,29.604873
3,0.863628,0.121489
4,1.852118,0.475642


In [41]:
k = len(variables)
k

2

In [42]:
adj_list_gs.sum(axis=1)/k

ID
0      0.615274
1      1.821357
2     17.728320
3      0.492558
4      1.163880
        ...    
80     6.629720
81     3.154587
82     3.872022
83     4.307689
84     1.080905
Length: 85, dtype: float64

In [43]:
import numpy as np
import pandas as pd
import warnings
from scipy import sparse
from scipy import stats
from sklearn.base import BaseEstimator
import libpysal as lp


class Local_Geary_MV(BaseEstimator):
    """Local Geary - Univariate"""

    def __init__(self, connectivity=None, inference=None):
        """
        Initialize a Local_Geary estimator

        Arguments
        ---------
        connectivity     : scipy.sparse matrix object
                           the connectivity structure describing the
                           relationships between observed units.
        inference        : str
                           describes type of inference to be used. options are
                           "chi-square" or "permutation" methods.

        Attributes
        ----------
        localG           : numpy array
                           Array of Local Geary values for each spatial unit.
        pval             : numpy array
                           P-values for inference based on either
                           "chi-square" or "permutation" methods.
        """

        self.connectivity = connectivity
        self.inference = inference

    def fit(self, variables):
        """
        Arguments
        ---------
        variables        : numpy.ndarray
                           array containing continuous data

        Returns
        -------
        the fitted estimator.

        Notes
        -----
        Technical details and derivations can be found in :cite:`Anselin1995`.

        Examples
        --------
        Guerry data replication GeoDa tutorial
        >>> import libpysal
        >>> import geopandas as gpd
        >>> guerry = lp.examples.load_example('Guerry')
        >>> guerry_ds = gpd.read_file(guerry.get_path('Guerry.shp'))
        >>> w = libpysal.weights.Queen.from_dataframe(guerry_ds)
        """
        self.variables = np.array(variables, dtype='float')

        w = self.connectivity
        w.transform = 'r'

        self.localG = self._statistic(variables, w)

        if self.inference is None:
            return self
        #elif self.inference == 'chi-square':
        #    if a != 2:
        #        warnings.warn(f'Chi-square inference assumes that a=2, but \
        #        a={a}. This means the inference will be invalid!')
        #    else:
        #        dof = 2/self.VarHi
        #        Zi = (2*self.Hi)/self.VarHi
        #        self.pval = 1 - stats.chi2.cdf(Zi, dof)
        else:
            raise NotImplementedError(f'The requested inference method \
            ({self.inference}) is not currently supported!')

        return self

    @staticmethod
    def _statistic(variables, w):
        # Caclulate z-scores for input variables
        zseries = [stats.zscore(i) for i in variables]
        # Define denominator adjustment
        k = len(variables)
        # Create focal and neighbor values
        adj_list = w.to_adjlist(remove_symmetric=False)
        zseries = [pd.Series(i, index=wq.id_order) for i in zseries]
        focal = [zseries[i].loc[adj_list.focal].values for
                 i in range(len(variables))]
        neighbor = [zseries[i].loc[adj_list.neighbor].values for
                    i in range(len(variables))]
        # Carry out local Geary calculation
        gs = sum(list(wq.weights.values()), []) * \
        (np.array(focal) - np.array(neighbor))**2
        # Reorganize data
        temp = pd.DataFrame(gs).T
        temp['ID'] = adj_list.focal.values
        adj_list_gs = temp.groupby(by='ID').sum()
        localG = adj_list_gs.sum(axis=1)/k
        
        return (localG)

In [44]:
functest = Local_Geary_MV(connectivity=wq).fit([x,y])
functest.localG

ID
0     0.153819
1     0.303560
2     2.954720
3     0.123140
4     0.387960
        ...   
80    1.657430
81    0.525764
82    0.645337
83    0.717948
84    0.216181
Length: 85, dtype: float64